<a href="https://colab.research.google.com/github/yukinaga/min_gen_agent_app/blob/main/section_3/02_voice__secretary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 音声でやりとりする秘書アプリ

このノートブックでは、音声で話しかけると答えてくれる秘書アプリを体験できます。  
OpenAI の Agents SDK を使い、自然な会話を通じてタスク管理や日付確認などを行います。  
ブラウザから直接マイクを使って対話できます。

## アプリの仕組み

###音声入力

マイクで話しかけると、音声データが録音されます。

録音した音声は OpenAI の音声認識モデル（gpt-4o-mini-transcribe）でテキストに変換されます。

### エージェントが理解・判断

テキスト化された内容を Agents SDK のエージェントに渡します。

エージェントは質問や命令を理解し、必要に応じて「タスクを追加する」「現在時刻を確認する」などの関数ツールを自動で呼び出します。

### 音声で回答

エージェントの回答テキストを 音声合成モデル（gpt-4o-mini-tts）が自然な日本語音声に変換します。

秘書が話しているように、音声で返答が再生されます。

### 使用するAI技術
OpenAI Agents SDK:	AIエージェントを作るためのPython SDK。関数ツールや会話メモリを簡単に扱えます。  
Speech-to-Text (STT):	音声をテキストに変換します。モデルは「gpt-4o-mini-transcribe」  
Text-to-Speech (TTS):	テキストを音声に変換します。モデルは「gpt-4o-mini-tts」

In [ ]:
# ============================================
# 🎙️ 音声でやりとりする秘書アプリ
# --------------------------------------------
# Google Colab 上で動作するデモ
# OpenAI Agents SDK を使って、音声で会話できる秘書を作ります。
# マイクで話しかけると：
#   1) 音声 → テキスト（文字起こし）
#   2) エージェントが回答を生成
#   3) 回答テキスト → 音声（合成音声で再生）
# を行います。
#
# 使い方：
# ① Colab の左サイドバー → 「🔑 Secrets」で
#    OPENAI_API_KEY を登録してください。
# ② このセルをそのまま実行します。
# ============================================

# ===== 必要なパッケージのインストール ===============================
!pip -q install --upgrade openai-agents openai gradio

# ===== インポートと初期設定 =========================================
import os, uuid, asyncio, tempfile
from google.colab import userdata
from openai import OpenAI
import gradio as gr
from agents import Agent, Runner, function_tool, SQLiteSession

# --- APIキーを Secrets から読み込み ---
api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY が見つかりません。左サイドバーの 🔑 Secrets で登録してください。")
os.environ["OPENAI_API_KEY"] = api_key

# OpenAI クライアントを初期化（環境変数を自動参照）
client = OpenAI()

# ===== 音声変換ユーティリティ ======================================
async def speech_to_text(audio_path: str) -> str:
    """音声ファイルをテキストに変換（Speech-to-Text）"""
    with open(audio_path, "rb") as f:
        tr = client.audio.transcriptions.create(
            model="gpt-4o-mini-transcribe",  # 高速で高精度な文字起こしモデル
            file=f,
        )
    return (tr.text or "").strip()

async def text_to_speech(text: str, voice: str = "alloy") -> str:
    """テキストを音声に変換（Text-to-Speech）"""
    out_path = os.path.join(tempfile.gettempdir(), f"reply_{uuid.uuid4().hex}.mp3")
    # 音声ファイルをストリーミングで保存（長文でも安全）
    with client.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts",
        voice=voice,
        input=text,
    ) as resp:
        resp.stream_to_file(out_path)
    return out_path

# ===== エージェントが使う関数ツール =================================
# この部分は「秘書が呼び出せる機能」を定義します。
TODO = []

@function_tool
def add_todo(task: str) -> str:
    """タスクを追加します。"""
    if not task.strip():
        return "空のタスクは追加できません。"
    TODO.append(task.strip())
    return f"タスクを追加: {task}（合計 {len(TODO)} 件）"

@function_tool
def list_todo() -> list[str]:
    """現在のタスク一覧を返します。"""
    return TODO

@function_tool
def clear_todo() -> str:
    """タスクをすべて削除します。"""
    TODO.clear()
    return "タスクをすべて削除しました。"

@function_tool
def now() -> str:
    """現在の日時（日本時間）を返します。"""
    from datetime import datetime, timezone, timedelta
    JST = timezone(timedelta(hours=9), name="JST")
    return datetime.now(JST).strftime("%Y-%m-%d %H:%M")

# ===== 秘書エージェントの設定 ======================================
secretary = Agent(
    name="Voice Secretary",
    instructions=(
        "あなたは音声でやりとりする日本語の秘書です。"
        "丁寧でわかりやすく、1〜3文で簡潔に答えてください。"
        "最後に『次のアクション』を1つ提案します。"
        "必要に応じて add_todo / list_todo / clear_todo / now を使ってください。"
    ),
    tools=[add_todo, list_todo, clear_todo, now],
)

# 会話の記憶を保持する軽量セッション
session = SQLiteSession("gradio_voice_secretary")

# ===== Gradio の処理関数 ============================================
GREETING = "こんにちは。秘書のエコです。ご用件をどうぞ。"

async def handle_interaction(audio_file, text_input, voice, messages):
    """音声またはテキスト入力を受け取り、エージェントの返答と音声を返す"""
    messages = messages or []

    # 1) 音声またはテキストを取得
    if audio_file:
        try:
            user_text = await speech_to_text(audio_file)
        except Exception as e:
            user_text = ""
            messages.append({"role": "assistant", "content": f"文字起こしエラー: {e}"})
    else:
        user_text = (text_input or "").strip()

    if not user_text:
        messages.append({"role": "assistant", "content": "音声またはテキストで話しかけてください。"})
        return messages, None

    messages.append({"role": "user", "content": user_text})

    # 2) エージェントに質問
    try:
        result = await Runner.run(secretary, input=user_text, session=session)
        bot_text = (result.final_output or "").strip()
    except Exception as e:
        bot_text = f"回答生成でエラーが発生しました: {e}"

    messages.append({"role": "assistant", "content": bot_text})

    # 3) テキストを音声に変換
    tts_path = None
    try:
        tts_path = await text_to_speech(bot_text, voice=voice)
    except Exception as e:
        messages.append({"role": "assistant", "content": f"音声合成に失敗しました: {e}"})

    return messages, tts_path

async def clear_all():
    """チャットとタスクを初期化"""
    TODO.clear()
    try:
        await session.clear_session()
    except Exception:
        pass
    return [{"role": "assistant", "content": GREETING}], None

# ===== Gradio UI の構築 =============================================
with gr.Blocks(title="音声秘書アプリ") as demo:
    gr.Markdown(
        "## 🎧 音声でやりとりする秘書アプリ\n"
        "- マイクで話しかけると、秘書がテキストと音声で返答します。\n"
        "- 例：「現在の日時を教えて」「タスクを一覧して」など。"
    )

    with gr.Row():
        audio_in = gr.Audio(sources=["microphone"], type="filepath", label="🎤 マイク入力")
        text_in = gr.Textbox(label="⌨️ テキスト入力（音声が使えないとき）",
                             placeholder="例）明日の午前中にA社に電話するタスクを追加して")

    with gr.Row():
        voice = gr.Dropdown(
            label="音声の種類（TTS Voice）",
            choices=["alloy", "shimmer", "nova", "onyx", "echo", "fable"],
            value="alloy"
        )
        send_btn = gr.Button("▶️ 送信", variant="primary")
        clear_btn = gr.Button("🧹 クリア")

    chat = gr.Chatbot(
        label="会話",
        type="messages",  # 推奨形式（警告回避）
        value=[{"role": "assistant", "content": GREETING}],
        height=360,
    )
    audio_out = gr.Audio(label="🔊 音声回答", autoplay=True)

    state = gr.State([{"role": "assistant", "content": GREETING}])

    # イベント設定
    send_btn.click(
        fn=handle_interaction,
        inputs=[audio_in, text_in, voice, state],
        outputs=[chat, audio_out]
    ).then(fn=lambda h, *_: h, inputs=[chat], outputs=[state])

    clear_btn.click(
        fn=clear_all,
        inputs=[],
        outputs=[chat, audio_out]
    ).then(fn=lambda h, *_: h, inputs=[chat], outputs=[state])

# ===== アプリの起動 =================================================
demo.launch()
